## BERT NOT FINE TUNED 
applied only to the test data. 

### Preproc

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import BertModel, BertTokenizer
import torch
from torch.nn.functional import cosine_similarity

In [4]:
dataset = load_dataset("PiC/phrase_similarity")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']
train_df = train_dataset.to_pandas()
validation_df = validation_dataset.to_pandas()
test_df = test_dataset.to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

### Model - Phrase

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_phrases(phrases):
    inputs = tokenizer(phrases, padding=True, truncation=True, return_tensors="pt")
    #embeddings from berts last hidden state
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def find_similarity(phrase1, phrase2):
    embeddings1 = encode_phrases([phrase1])
    embeddings2 = encode_phrases([phrase2])
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity.item()


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
similarity = []
for _,row in test_df.iterrows():
  similarity.append(find_similarity(row['phrase1'], row['phrase2']))

In [11]:
test_df['cosine_similarity'] = similarity


In [36]:
count = 0;
thres = 0.75 #similar if cosine similarity greater than 0.75, not similar if less.
for _,row in test_df.iterrows():
  if (row['cosine_similarity']>=thres and row['label']==1) or (row['cosine_similarity']< thres and row['label']==0):
    count +=1
accuracy = count/len(similarity)
print(f'The accuracy of BERT without fine tuning in calculating similarity is, {accuracy* 100} %')

The accuracy of BERT without fine tuning in calculating similarity is, 51.1 %


now if we do the same thing and encode the sentences in it rather than the phrases, lets check our accuracy. We then give more context to our bert model to analyze if they are similar or not.

In [37]:
similarity_sent = []
for _,row in test_df.iterrows():
  similarity_sent.append(find_similarity(row['sentence1'], row['sentence2']))

In [39]:
test_df['cosine_similarity_s'] = similarity_sent

In [42]:
count = 0;
thres = 0.75 #similar if cosine similarity greater than 0.75, not similar if less.
for _,row in test_df.iterrows():
  if (row['cosine_similarity_s']>=thres and row['label']==1) or (row['cosine_similarity_s']< thres and row['label']==0):
    count +=1
accuracy_s = count/len(similarity_sent)
print(f'The accuracy of BERT without fine tuning in calculating similarity is, {accuracy_s* 100} %')

The accuracy of BERT without fine tuning in calculating similarity is, 50.05 %


we see that this doesnt change the model too much.

### Model and Prepoc: Sentence

BERT no fine tuning for sentence embeddings:

In [44]:
sentence = load_dataset("paws", "labeled_final")

test_sentence = sentence['test']
val = 1000
test_sentence[:val]
test_sent_df = test_sentence.to_pandas()

In [45]:
cos_similarity_sent = []
for _,row in test_sent_df.iterrows():
  cos_similarity_sent.append(find_similarity(row['sentence1'], row['sentence2']))

In [46]:
test_sent_df['similarity'] = cos_similarity_sent

In [47]:
count = 0;
thres = 0.5 #similar if cosine similarity greater than 0.75, not similar if less.
for _,row in test_sent_df.iterrows():
  if (row['similarity']>=thres and row['label']==1) or (row['similarity']< thres and row['label']==0):
    count +=1
accuracy = count/len(cos_similarity_sent)
print(f'The accuracy of BERT without fine tuning in calculating similarity is, {accuracy* 100} %')

The accuracy of BERT without fine tuning in calculating similarity is, 44.2 %


The similarity here is suprisingly low, hence we need to fine tune it.